<a href="https://colab.research.google.com/github/OumniaArkonte/Real-Estate/blob/main/Reinforcement_Learning_Sudoku_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth transformers peft datasets trl

  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached triton-3.5.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.24.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl (899.7 MB)
Using cached triton-3.5.1-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.5 MB)
Using cached torchvision-0.24.1-cp312-cp312-manylinux_2_28_x86_64.whl (8.0 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling triton-3.5.0:
      Successfully uninstalled triton-3.5.0
  Attempting uninstall: torch
    Found existing installation: torch 2.9.0
    Uninstalling torch-2.9.0:
      Successfully uninstalled torch-2.9.0
  Attempting uninstall: 

### Unsloth

In [7]:
!pip uninstall -y vllm
!pip install vllm==0.3.3

  Using cached vllm-0.3.3.tar.gz (315 kB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
pip install vllm

  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.9 kB)
  Using cached triton-3.5.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl (899.7 MB)
Using cached torchvision-0.24.0-cp312-cp312-manylinux_2_28_x86_64.whl (8.1 MB)
Using cached triton-3.5.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.1
    Uninstalling triton-3.5.1:
      Successfully uninstalled triton-3.5.1
  Attempting uninstall: torch
    Found existing installation: torch 2.9.1
    Uninstalling torch-2.9.1:
      Successfully uninstalled torch-2.9.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.24.1
    Uninstalling torchvision-0.24.1:
      Successfully uninstalled torchvision-0.24

In [4]:
from unsloth import FastLanguageModel

model_name = "unsloth/mistral-7b-bnb-4bit"  # modèle official Unsoth
# ou un modèle HuggingFace existant comme:
# model_name = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    load_in_4bit = True,
    fast_inference = False,
    gpu_memory_utilization = 0.9,
)

==((====))==  Unsloth 2025.11.6: Fast Mistral patching. Transformers: 4.57.2. vLLM: 0.12.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

To do efficient RL, we will use [LoRA](https://arxiv.org/abs/2106.09685), which allows us to only add 1 to 5% of extra weights to the model for finetuning purposes. This allows us to save memory usage by over 60%, and yet it retains good accuracy.

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,  # lora_rank
    lora_alpha = 128,
    use_gradient_checkpointing = "unsloth", # Pour gros modèles/Vram faible
    random_state = 42,
)

Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/sample_data/dataset_grpo_resume.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
from trl import GRPOTrainer, GRPOConfig

config = GRPOConfig(
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    max_steps=20,
    gradient_accumulation_steps=4,
    fp16=True,          # ou False selon GPU
    logging_steps=10,   # affiche tous les 10 steps
    output_dir="outputs"
)

# [IMPORTANT] Vérifie rapidement ton dataset (affiche une ligne pour checker les champs)
print(dataset[0])

Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8
{'prompt': 'Texte original : La réunion hebdomadaire de l’équipe marketing a porté sur les performances de la campagne digitale lancée en septembre. Les indicateurs montrent une hausse de 18 % du taux de clic, mais le taux de conversion reste en dessous des objectifs. Une nouvelle stratégie de ciblage sera testée dès la semaine prochaine.', 'response': '{"Titre":"Réunion Marketing - Campagne Digitale","Résumé":"La campagne affiche une forte progression du taux de clic, mais une conversion inférieure aux attentes.","Décisions":["Tester une nouvelle stratégie de ciblage"],"Actions":["Lancer un test dès la semaine prochaine"]}', 'reward': 1.0}


In [22]:
def reward_func(prompts, completions, completion_ids=None, **kwargs):
    # The GRPOTrainer passes prompts, completions, etc.
    # Since the dataset example had reward: 1.0, we will return 1.0 for each completion.
    return [1.0] * len(completions)

In [23]:
trainer = GRPOTrainer(
    model=model,
    args=config,
    tokenizer=tokenizer,
    train_dataset=dataset,
    reward_funcs=reward_func  # <-- c'est cet argument qu'il fallait rajouter !
)
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 4 | Total steps = 20
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 167,772,160 of 7,409,504,256 (2.26% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / reward_func / mean,rewards / reward_func / std
10,0.000100,1.000000,0.000000,225.990625,43.000000,256.000000,0.806250,113.793159,43.000000,214.600000,0,0,0,0,0,0.122694,1.000000,0.000000
20,0.000500,1.000000,0.000000,148.656250,17.400000,240.600000,0.453125,79.651343,17.400000,212.800000,No Log,No Log,No Log,No Log,No Log,0.454287,1.000000,0.000000


profiling/Time taken: UnslothGRPOTrainer._calculate_rewards,█▇▆█▂▇▁▄▄▂▃█▂▂█▁▁▂▂▁▄▁▂▂▁▂
profiling/Time taken: UnslothGRPOTrainer._prepare_inputs,▁▁█▁█▁▁▁▁▁▁██▁▁▁█▁▁█▁▁▁▁██▁▁█▁▄▁▁█▁█▁█▁▁
profiling/Time taken: UnslothGRPOTrainer.reward_func,▂█▃▅▂▅▁▂▄▂▃▅▂▂▆▁▁▂▂▁▅▁▂▂▁▄
profiling/Time taken: UnslothGRPOTrainer.transformers.generate,██████████████████▇█▁▇█████
train/completion_length,█▁
train/completions/clipped_ratio,█▁
train/completions/max_length,█▁
train/completions/max_terminated_length,█▁
train/completions/mean_length,█▁
train/completions/mean_terminated_length,█▁
+14,...


TrainOutput(global_step=20, training_loss=0.00028849068912677466, metrics={'train_runtime': 3154.5344, 'train_samples_per_second': 0.203, 'train_steps_per_second': 0.006, 'total_flos': 0.0, 'train_loss': 0.00028849068912677466})

In [24]:
model.save_pretrained("output_dir")
tokenizer.save_pretrained("output_dir")
# OU (selon la doc unsloth/peft et ce que tu veux sauvegarder)

# Pour sauvegarder les poids LoRA uniquement
from peft import PeftModel
PeftModel.from_pretrained(model, "output_dir")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): MistralForCausalLM(
          (model): MistralModel(
            (embed_tokens): Embedding(32000, 4096, padding_idx=0)
            (layers): ModuleList(
              (0-31): 32 x MistralDecoderLayer(
                (self_attn): MistralAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
         

<a name="Inference"></a>
# Inference
Now let's try the model we just trained!

In [26]:
import torch

prompt = """Texte original : Une réunion projet a souligné les retards sur le livrable X, mais noté l’achèvement de plusieurs tâches R&D. Une feuille de route révisée a été adoptée. Résume ce texte de façon structurée (titre, résumé, actions, décisions...) en JSON."""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=1.0,
        do_sample=True,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Texte original : Une réunion projet a souligné les retards sur le livrable X, mais noté l’achèvement de plusieurs tâches R&D. Une feuille de route révisée a été adoptée. Résume ce texte de façon structurée (titre, résumé, actions, décisions...) en JSON.
\begin{json}[01-10-2020]
				{
				  "title": "Réunion du vendredi 10 octobre 2020",
				  "summary": "Mise au courant des tâches. Provisoirement, le protocole de lancement du programme Pouet Pouet est suspendu, jusqu'à validation du système de contrôle de données SCD.",
				  "actions": [
						 {
						  "type": "todo",
						  "name": "Etude du protocole de transfert des commandes LES-FDS et PPS-CDS"
						 }
					],
			


<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
from unsloth import FastLanguageModel

# Après avoir entraîné et LoRA adapté
# (Supposons que tu as : model, tokenizer déjà configuré)

# Fusionne en 16bit
model.save_pretrained_merged("model_dir_16bit", tokenizer, save_method="merged_16bit")

# (ou) Fusionne en 4bit
model.save_pretrained_merged("model_dir_4bit", tokenizer, save_method="merged_4bit")

# (option) Upload sur Hugging Face
model.push_to_hub_merged(
    "username/mon-modele-fusionne",
    tokenizer,
    save_method="merged_16bit",
    token="hf_xxx..."
)

# (option) Sauvegarde/adaptateur LoRA seulement (flexible si tu veux faire du multi-task peft, etc)
model.save_pretrained("model_lora_only")
tokenizer.save_pretrained("model_lora_only")